In [25]:
import numpy as np
import scipy as sp
from importlib import reload
import neuraltda.topology2 as tp2
import neuraltda.plotting as tpplt
import neuraltda.spikeutils as su
import glob
import os
import matplotlib.pyplot as plt
import h5py as h5
import tqdm
%matplotlib inline
import datetime

# Setup figure save path
daystr = datetime.datetime.now().strftime('%Y%m%d')
figsavepth = '/home/brad/DailyLog/'+daystr+'/'
print(figsavepth)

/home/brad/DailyLog/20181106/


In [3]:
# Set up birds and block_paths
birds = ['B1083', 'B1056', 'B1235', 'B1075']
bps = {'B1083': '/home/brad/krista/B1083/P03S03/', 'B1075': '/home/brad/krista/B1075/P01S03/',
       'B1235': '/home/brad/krista/B1235/P02S01/', 'B1056': '/home/brad/krista/B1056/klusta/phy020516/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/',
       'B1056': '/home/brad/krista/B1056/klusta/phy020516/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/'}

# Binning Parameters
windt = 10.0                      # milliseconds
dtovr = 0.0*windt                 # milliseconds
segment_info = [0, 0]             # use full Trial
cluster_group = ['Good']          # use just good clusters
comment = 'Flicker'        # BootStrap Populations
bdfs = {}                         # Dictionary to store bdf

# Betti Parameters
analysis_id = 'BC'                # Generic lable
nperms = None                     # No permutation for each trial
ncellsperm = 33                   # Subsample 33 neurons from the population
dims = [0, 1, 2, 3]               # We want first four betti numbers
thresh = 4.0                     # Activation threshold
twin = np.linspace(0, 6000, 1000) # milliseconds

In [4]:
# Loop through each bird in our list and bin the data
for bird in birds:
    block_path = bps[bird]
    bfdict = tp2.dag_bin(block_path, windt, segment_info, cluster_group=cluster_group, dt_overlap=dtovr, comment=comment)
    bdf = glob.glob(os.path.join(bfdict['raw'], '*.binned'))[0]
    print(bdf)
    bdfs[bird] = bdf

Data Not already binned
Nreps = 20
skip = 200
dur = 120000.0
nwins = 600
Nreps = 20
skip = 200
dur = 120000.0
nwins = 600
Nreps = 20
skip = 200
dur = 120000.0
nwins = 600
Nreps = 20
skip = 200
dur = 120000.0
nwins = 600
Nreps = 20
skip = 200
dur = 120000.0
nwins = 600
Nreps = 20
skip = 200
dur = 120000.0
nwins = 600
Nreps = 20
skip = 200
dur = 120000.0
nwins = 600
Nreps = 20
skip = 200
dur = 120000.0
nwins = 600
/home/brad/krista/B1083/P03S03/binned_data/win-10.0_dtovr-0.0_seg-0-0-Flicker/20181106T221701Z-10.0-0.0.binned
Data Not already binned
Nreps = 20
skip = 312
dur = 206722.0
nwins = 663
Nreps = 20
skip = 312
dur = 206723.0
nwins = 663
Nreps = 20
skip = 312
dur = 206719.0
nwins = 663
Nreps = 20
skip = 312
dur = 206719.0
nwins = 663
Nreps = 20
skip = 312
dur = 206722.0
nwins = 663
Nreps = 20
skip = 312
dur = 206719.0
nwins = 663
Nreps = 20
skip = 312
dur = 206718.0
nwins = 663
Nreps = 20
skip = 312
dur = 206719.0
nwins = 663
Nreps = 20
skip = 312
dur = 206719.0
nwins = 663
Nreps = 

In [74]:
def get_cg_times(bdf):
    with h5.File(bdf) as f:
        cg_times_all_stim = {}
        for stim in tqdm.tqdm(f.keys()):
            poptens = f[stim]['pop_tens']
            (ncell, nwin, ntrial) = poptens.shape
            for trial in range(ntrial):
                cg_times = {}
                trial_cellgroups = tp2.calc_cell_groups(poptens[:, :, trial], np.arange(ncell), thresh)
                #print(trial_cellgroups)
                unique_cell_groups = list(set([tuple(x[1]) for x in trial_cellgroups]))
                for ucg in unique_cell_groups:
                    times = [x[0] for x in trial_cellgroups if tuple(x[1]) == ucg]
                    cg_times[ucg] = times
            cg_times_all_stim[stim] = cg_times
    return cg_times_all_stim

def compute_cg_timecourse(cg_times, maxt, tau):
    cg_timecourse = {}
    FILT_LENGTH = 100
    times = np.arange(FILT_LENGTH)
    filt = np.exp(-times/tau)
    for cg in cg_times.keys():
        timecourse = np.zeros(maxt)
        timecourse[cg_times[cg]] = 1
        timecourse = np.convolve(timecourse, filt, 'same')
        cg_timecourse[cg] = timecourse
    return cg_timecourse
        
def compute_flicker_complex(cg_timecourse, maxt, cg_thresh):
    cgs = list(cg_timecourse.keys())
    timecourse_mat = np.empty((0, maxt))
    for cg in cgs:
        timecourse_mat = np.vstack([timecourse_mat, cg_timecourse[cg]])
    flicker_complex = np.greater(timecourse_mat, cg_thresh)
    return (flicker_complex, np.array(cgs))

import networkx as nx
def compute_graph_sequence(flicker_complex, cell_groups):
    # Build all simplices
    cg_graphs = {}
    for cg in cell_groups:
        cg_graphs[cg] = nx.complete_graph(cg)
        
    #stitch together
    (ncg, nt) = flicker_complex.shape
    graph_sequence = []
    for t in range(nt):
        active_cgs = cell_groups[flicker_complex[:, t]]
        print(active_cgs)
        g = nx.compose_all([cg_graphs[x] for x in active_cgs])
        graph_sequence.append(g)
    return graph_sequence

In [28]:
test = get_cg_times(bdfs['B1083'])

100%|██████████| 8/8 [00:25<00:00,  3.15s/it]


In [48]:
tc = compute_cg_timecourse(test['L_40k'], 600, 5)

In [75]:
acg, cgs = compute_flicker_complex(tc, 600, 0.2)

In [76]:
gs  = compute_graph_sequence(acg, cgs)

[(21, 44) (21, 31) (13, 21, 31) (42, 95, 99) (15, 26, 41, 42, 44, 65)
 (11, 15, 63) (15, 19, 59, 93) (3, 4, 12, 13, 21, 44, 93) (42, 63, 93, 99)]
[(14, 21, 59) (21, 44) (21, 31) (42, 95, 99) (15, 26, 41, 42, 44, 65)
 (11, 15, 63) (15, 19, 59, 93) (3, 4, 12, 13, 21, 44, 93) (42, 63, 93, 99)]
[(14, 21, 59) (21, 44) (21, 31) (42, 95, 99) (15, 26, 41, 42, 44, 65)
 (11, 15, 63) (21, 42, 99) (15, 19, 59, 93) (42, 63, 93, 99)]
[(14, 21, 59) (21, 44) (42, 95, 99) (15, 26, 41, 42, 44, 65) (11, 15, 63)
 (21, 42, 99) (15, 19, 59, 93) (42, 63, 93, 99) (19, 44, 63)]
[(59, 63) (14, 21, 59) (21, 44) (42, 95, 99) (11, 15, 63) (21, 42, 99)
 (15, 19, 59, 93) (42, 63, 93, 99) (19, 44, 63)]
[(10, 13, 21, 22, 37, 42, 44, 92, 99) (59, 63) (14, 21, 59) (21, 44)
 (42, 95, 99) (11, 15, 63) (21, 42, 99) (15, 19, 59, 93) (19, 44, 63)]
[(10, 13, 21, 22, 37, 42, 44, 92, 99) (59, 63) (14, 21, 59) (42, 95, 99)
 (11, 15, 63) (21, 42, 99) (15, 19, 59, 93) (19, 44, 63)
 (0, 6, 21, 63, 94)]
[(10, 13, 21, 22, 37, 42, 44,

StopIteration: 

In [70]:
acg[:, 233]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,